In [ ]:
import sympy as sym
import numpy as np
def integrate (f, xt, dt, tt):

  k1 = dt*f(xt, tt)
  k2 = dt*f(xt+k1/2., tt+dt/2.)
  k3 = dt*f(xt+k2/2., tt+dt/2.)
  k4 = dt*f(xt+k3, tt+dt)
  new_xt = xt + (1/6.)*(k1 + 2.0*k2 + 2.0*k3 + k4)
  return new_xt


def cmat(theta,x,y):

  mat = sym.Matrix([[sym.cos(theta), -sym.sin(theta),0, x],
      [sym.sin(theta), sym.cos(theta),0, y],
      [0,0, 1,0],
      [0,0, 0,1]])
  return mat
def hat(v):

  mat = sym.Matrix([0, -v[2],v[1]],
                    [v[2],0, -v[0]],
                    [-v[1],v[0],0])
  return mat
def unhat(m):
  v = sym.Matrix([m[0,3],m[1,3],m[2,3],m[2,1],m[0,2],m[1,0]])
  return v

def invg(g):

  R = g[0:3,0:3]
  p = g[0:3,3:4]

  Rt = R.T



  pin = -Rt*p

  ginv = sym.Matrix([[Rt[0,0],Rt[0,1],Rt[0,2],pin[0]],
                    [Rt[1,0],Rt[1,1],Rt[1,2],pin[1]],
                    [Rt[2,0],Rt[2,1],Rt[2,2],pin[2]],
                    [0,0,0,1]])

  return ginv

In [ ]:
t = sym.symbols('t')

xb = sym.Function(r'x_box')(t)
yb = sym.Function(r'y_box')(t)
xj = sym.Function(r'x_jack')(t)
yj = sym.Function(r'y_jack')(t)
thetaj = sym.Function(r'\theta_jack')(t)
thetab = sym.Function(r'\theta_box')(t)

# Frame A' to Box
gwb = cmat(0,xb,yb)


gwbs = cmat(thetab,0,0)*gwb

#Shortcut frame A to Jack
gwjs = (cmat(0,xj,yj))*cmat(thetaj,0,0)


#Shortcut frame A to Box
gwbs = gwb*cmat(thetab,0,0)


#velocity of block
Vb_b = unhat(invg(gwbs)*gwbs.diff(t))

Vb_j = unhat(invg(gwjs)*gwjs.diff(t))




In [ ]:
Vb_j

Matrix([
[                sin(\theta_jack(t))*Derivative(y_jack(t), t) + cos(\theta_jack(t))*Derivative(x_jack(t), t)],
[               -sin(\theta_jack(t))*Derivative(x_jack(t), t) + cos(\theta_jack(t))*Derivative(y_jack(t), t)],
[                                                                                                          0],
[                                                                                                          0],
[                                                                                                          0],
[sin(\theta_jack(t))**2*Derivative(\theta_jack(t), t) + cos(\theta_jack(t))**2*Derivative(\theta_jack(t), t)]])

In [ ]:
#jack if theta relative to box

bbb = gwbs*gwjs

In [ ]:
Lb = 6
mb = 350
Jb = 4*mb*Lb**2
g = 9.81
Lj = 1
mj = 30
Jj = mj*Lj**2
inertia_b = sym.Matrix([[mb,0,0,0,0,0],
                  [0,mb,0,0,0,0],
                  [0,0,4*mb,0,0,0],
                  [0,0,0,0,0,0],
                  [0,0,0,0,0,0],
                  [0,0,0,0,0,Jb]])


inertia_j = sym.Matrix([[4*mj,0,0,0,0,0],
                  [0,4*mj,0,0,0,0],
                  [0,0,4*mj,0,0,0],
                  [0,0,0,0,0,0],
                  [0,0,0,0,0,0],
                  [0,0,0,0,0,Jj]])

In [ ]:
KE = 0.5*(Vb_b.T*inertia_b*Vb_b)[0] +0.5*(Vb_j.T*inertia_j*Vb_j)[0]
V = 4*mb*g*(gwbs[0:3,3:4][1]) + 4*mj*g*(gwjs[0:3,3:4][1])

In [ ]:
mb*g*(gwbs[0:3,3:4][1])

3433.5*y_box(t)

In [ ]:
mb*g*(gwjs[0:3,3:4][1])

3433.5*y_jack(t)

In [ ]:
(gwjs[0:3,3:4][1])

y_jack(t)

In [ ]:
sym.simplify(KE)

25200.0*Derivative(\theta_box(t), t)**2 + 15.0*Derivative(\theta_jack(t), t)**2 + 175.0*Derivative(x_box(t), t)**2 + 60.0*Derivative(x_jack(t), t)**2 + 175.0*Derivative(y_box(t), t)**2 + 60.0*Derivative(y_jack(t), t)**2

In [ ]:
L = KE - V
L = L.simplify()

In [ ]:
L

-13734.0*y_box(t) - 1177.2*y_jack(t) + 25200.0*Derivative(\theta_box(t), t)**2 + 15.0*Derivative(\theta_jack(t), t)**2 + 175.0*Derivative(x_box(t), t)**2 + 60.0*Derivative(x_jack(t), t)**2 + 175.0*Derivative(y_box(t), t)**2 + 60.0*Derivative(y_jack(t), t)**2

In [ ]:
t

t

In [ ]:
q  = sym.Matrix([xb, yb, thetab,xj,yj,thetaj])
qd = q.diff(t)
qd2 = qd.diff(t)
dLdq = L.diff(q)
dLdqdot = L.diff(qd)
E_Leq =  dLdqdot.diff(t) - dLdq
E_Leq = sym.simplify(E_Leq)

In [ ]:
E_Leq

Matrix([
[          350.0*Derivative(x_box(t), (t, 2))],
[350.0*Derivative(y_box(t), (t, 2)) + 13734.0],
[   50400.0*Derivative(\theta_box(t), (t, 2))],
[         120.0*Derivative(x_jack(t), (t, 2))],
[120.0*Derivative(y_jack(t), (t, 2)) + 1177.2],
[     30.0*Derivative(\theta_jack(t), (t, 2))]])

In [ ]:
Fy = 0.99*mb*g
Fth = 30000*(sym.sin(np.pi*t/4))
Fx = 300*(sym.cos(np.pi*t/5))
F = sym.Matrix([0,Fy,Fth,0,0,0])
eqns = sym.Eq(E_Leq,sym.simplify(F))
eqqns = sym.simplify(eqns)

solns[0][qd2[1]]  = 0

In [ ]:
solns = sym.solve(eqqns, qd2, dict=True)
solns[0][qd2[1]]=0
display(solns)
xddot_b = sym.lambdify([*q, *qd, t], solns[0][qd2[0]])
yddot_b = sym.lambdify([*q, *qd, t], solns[0][qd2[1]])
tddot_b = sym.lambdify([*q, *qd, t], solns[0][qd2[2]])
xddot_j = sym.lambdify([*q, *qd, t], solns[0][qd2[3]])
yddot_j = sym.lambdify([*q, *qd, t], solns[0][qd2[4]])
tddot_j = sym.lambdify([*q, *qd, t], solns[0][qd2[5]])

[{Derivative(x_box(t), (t, 2)): 0.0,
  Derivative(y_box(t), (t, 2)): 0,
  Derivative(\theta_box(t), (t, 2)): 0.595238095238095*sin(0.785398163397448*t),
  Derivative(x_jack(t), (t, 2)): 0.0,
  Derivative(y_jack(t), (t, 2)): -9.81000000000000,
  Derivative(\theta_jack(t), (t, 2)): 0.0}]

In [ ]:
qd2

Matrix([
[      Derivative(x_box(t), (t, 2))],
[      Derivative(y_box(t), (t, 2))],
[ Derivative(\theta_box(t), (t, 2))],
[     Derivative(x_jack(t), (t, 2))],
[     Derivative(y_jack(t), (t, 2))],
[Derivative(\theta_jack(t), (t, 2))]])

In [ ]:
qd2[1]

Derivative(y_box(t), (t, 2))

In [ ]:
def dyn(s,t):
  display(s)
  display(t)

  return np.array([
      s[6],
      s[7],
      s[8],
      s[9],
      s[10],
      s[11],
      xddot_b(*s,t),
      yddot_b(*s,t),
      tddot_b(*s,t),
      xddot_j(*s,t),
      yddot_j(*s,t),
      tddot_j(*s,t)
  ])

In [ ]:
xbminus = sym.symbols('x_box-')
ybminus = sym.symbols('y_box-')
thetabminus = sym.symbols(r'\theta_box-')

xjminus = sym.symbols('x_jack-')
yjminus = sym.symbols('y_jack-')
thetajminus = sym.symbols(r'\theta_jack-')

xbdotminus = sym.symbols('xdot_box-')
ybdotminus = sym.symbols('ydot_box-')
thetabdotminus = sym.symbols(r'\dot\theta_box-')

xjdotminus = sym.symbols('xdot_jack-')
yjdotminus = sym.symbols('ydot_jack-')
thetajdotminus = sym.symbols(r'\dot\theta_jack-')

In [ ]:
xbplus = sym.symbols('x_box+')
ybplus = sym.symbols('y_box+')
thetabplus = sym.symbols(r'\theta_box+')

xjplus = sym.symbols('x_jack+')
yjplus = sym.symbols('y_jack+')
thetajplus = sym.symbols(r'\theta_jack+')

xbdotplus = sym.symbols('xdot_box+')
ybdotplus = sym.symbols('ydot_box+')
thetabdotplus = sym.symbols(r'\dot\theta_box+')

xjdotplus = sym.symbols('xdot_jack+')
yjdotplus = sym.symbols('ydot_jack+')
thetajdotplus = sym.symbols(r'\dot\theta_jack+')

In [ ]:
Ham = dLdqdot.dot(qd) - L
Ham = sym.simplify(Ham)
Ham_m = Ham.subs({qd[0]:xbdotminus,qd[1]:ybdotminus,qd[2]:thetabdotminus,
                  qd[3]:xjdotminus,qd[4]:yjdotminus,qd[5]:thetajdotminus,
                  q[0]:xbminus,q[1]:ybminus,q[2]:thetabminus,
                  q[3]:xjminus,q[4]:yjminus,q[5]:thetajminus})

In [ ]:
Ham

13734.0*y_box(t) + 1177.2*y_jack(t) + 25200.0*Derivative(\theta_box(t), t)**2 + 15.0*Derivative(\theta_jack(t), t)**2 + 175.0*Derivative(x_box(t), t)**2 + 60.0*Derivative(x_jack(t), t)**2 + 175.0*Derivative(y_box(t), t)**2 + 60.0*Derivative(y_jack(t), t)**2

In [ ]:
Ham_m

25200.0*\dot\theta_box-**2 + 15.0*\dot\theta_jack-**2 + 175.0*xdot_box-**2 + 60.0*xdot_jack-**2 + 13734.0*y_box- + 1177.2*y_jack- + 175.0*ydot_box-**2 + 60.0*ydot_jack-**2

In [ ]:

Ham_p = Ham_m.subs({xbdotminus:xbdotplus,ybdotminus:ybdotplus,thetabdotminus:thetabdotplus,
                  xjdotminus:xjdotplus,yjdotminus:yjdotplus,thetajdotminus:thetajdotplus})

In [ ]:
dLdqdot = sym.simplify(dLdqdot)

dLdqdotm = dLdqdot.subs({qd[0]:xbdotminus,qd[1]:ybdotminus,qd[2]:thetabdotminus,
                  qd[3]:xjdotminus,qd[4]:yjdotminus,qd[5]:thetajdotminus,
                  q[0]:xbminus,q[1]:ybminus,q[2]:thetabminus,
                  q[3]:xjminus,q[4]:yjminus,q[5]:thetajminus})

In [ ]:
dLdqdotm

Matrix([
[        350.0*xdot_box-],
[        350.0*ydot_box-],
[50400.0*\dot\theta_box-],
[       120.0*xdot_jack-],
[       120.0*ydot_jack-],
[  30.0*\dot\theta_jack-]])

In [ ]:
dLdqdotp = dLdqdotm.subs({xbdotminus:xbdotplus,ybdotminus:ybdotplus,thetabdotminus:thetabdotplus,
                  xjdotminus:xjdotplus,yjdotminus:yjdotplus,thetajdotminus:thetajdotplus})

In [ ]:
L

-13734.0*y_box(t) - 1177.2*y_jack(t) + 25200.0*Derivative(\theta_box(t), t)**2 + 15.0*Derivative(\theta_jack(t), t)**2 + 175.0*Derivative(x_box(t), t)**2 + 60.0*Derivative(x_jack(t), t)**2 + 175.0*Derivative(y_box(t), t)**2 + 60.0*Derivative(y_jack(t), t)**2

In [ ]:
dLdqdot

Matrix([
[       350.0*Derivative(x_box(t), t)],
[       350.0*Derivative(y_box(t), t)],
[50400.0*Derivative(\theta_box(t), t)],
[      120.0*Derivative(x_jack(t), t)],
[      120.0*Derivative(y_jack(t), t)],
[  30.0*Derivative(\theta_jack(t), t)]])

In [ ]:
gwbr = gwbs*cmat(0,Lb, 0)
gwbb = gwbs*cmat(0,0,-Lb)
gwbl = gwbs*cmat(0,-Lb, 0)
gwbt = gwbs*cmat(0,0, Lb)

gwjr = gwjs*cmat(0,Lj, 0)
gwjb = gwjs*cmat(0,0,-Lj)
gwjl = gwjs*cmat(0,-Lj, 0)
gwjt = gwjs*cmat(0,0, Lj)

In [ ]:
gwbr_jr = invg(gwbr) * gwjr
gwbr_jb = invg(gwbr) * gwjb
gwbr_jl = invg(gwbr) * gwjl
gwbr_jt = invg(gwbr) * gwjt

gwbb_jr = invg(gwbb) * gwjr
gwbb_jb = invg(gwbb) * gwjb
gwbb_jl = invg(gwbb) * gwjl
gwbb_jt = invg(gwbb) * gwjt

gwbl_jr = invg(gwbl) * gwjr
gwbl_jb = invg(gwbl) * gwjb
gwbl_jl = invg(gwbl) * gwjl
gwbl_jt = invg(gwbl) * gwjt

gwbt_jr = invg(gwbt) * gwjr
gwbt_jb = invg(gwbt) * gwjb
gwbt_jl = invg(gwbt) * gwjl
gwbt_jt = invg(gwbt) * gwjt

In [ ]:
phi = sym.Matrix([gwbr_jr[3],gwbr_jb[3],gwbr_jl[3],gwbr_jt[3],
              gwbb_jr[7],gwbb_jb[7],gwbb_jl[7],gwbb_jt[7],
              gwbl_jr[3],gwbl_jb[3],gwbl_jl[3],gwbl_jt[3],
              gwbt_jr[7],gwbt_jb[7],gwbt_jl[7],gwbt_jt[7]])

In [ ]:
gwbr_jr[3]

-(x_box(t) + 6*cos(\theta_box(t)))*cos(\theta_box(t)) + (x_jack(t) + cos(\theta_jack(t)))*cos(\theta_box(t)) - (y_box(t) + 6*sin(\theta_box(t)))*sin(\theta_box(t)) + (y_jack(t) + sin(\theta_jack(t)))*sin(\theta_box(t))

In [ ]:
phi = sym.simplify(phi)
dphi = phi.jacobian([q])

In [ ]:
dphi = dphi.subs({qd[0]:xbdotminus,qd[1]:ybdotminus,qd[2]:thetabdotminus,
                  qd[3]:xjdotminus,qd[4]:yjdotminus,qd[5]:thetajdotminus,
                  q[0]:xbminus,q[1]:ybminus,q[2]:thetabminus,
                  q[3]:xjminus,q[4]:yjminus,q[5]:thetajminus})


In [ ]:
impact_eqns = []

lhs = sym.Matrix([dLdqdotp[0]-dLdqdotm[0], dLdqdotp[1]-dLdqdotm[1],
       dLdqdotp[2]-dLdqdotm[2], dLdqdotp[3]-dLdqdotm[3],
       dLdqdotp[4]-dLdqdotm[4], dLdqdotp[5]-dLdqdotm[5],
       Ham_p-Ham_m
       ])

In [ ]:
lmbd = sym.symbols(r'lambda')


for i in range(phi.shape[0]):
   rhs = sym.Matrix([lmbd*dphi[i,0],
                     lmbd*dphi[i,1],
                     lmbd*dphi[i,2],
                     lmbd*dphi[i,3],
                     lmbd*dphi[i,4],
                     lmbd*dphi[i,5],
                     0])
   impact_eqns.append(sym.simplify(sym.Eq(lhs,rhs)))




In [ ]:
v = [*q,*qd]

In [ ]:
phif = sym.lambdify(v,phi)

In [ ]:
dum_list = {xbdotplus,ybdotplus,thetabdotplus,
            xjdotplus,yjdotplus,thetajdotplus}

In [ ]:
def impact_condition_die_jack(s,psi):
  p = phif(*s)
  for i in range(p.shape[0]):
    if(p[i]> -0.1) and (p[i]<0.1):
      return(True,i)
  return(False,None)

In [ ]:
def impact_update_die_jack(s,eqns,vals):

  sym_eqns = eqns.subs({xbminus:s[0],ybminus:s[1],thetabminus:s[2],
                  xjminus:s[3] ,yjminus: s[4],thetajminus: s[5],
                 xbdotminus: s[6],ybdotminus: s[7],thetabdotminus: s[8],
                  xjdotminus: s[9],yjdotminus: s[10],thetajdotminus: s[11]}
                  )
  solns = sym.solve(sym_eqns,[xbdotplus,ybdotplus,thetabdotplus,
                              xjdotplus,yjdotplus,thetajdotplus, lmbd], dict=True )

  if len(solns)==1:
    print("only one soln")
  else:
    for sol in solns:
      an = sol[lmbd]
      if abs(an) < 1e-06:
        pass

      else:
        return np.array([
            s[0],
            s[1],
            s[2],
            s[3],
            s[4],
            s[5],
            float(sym.N(sol[vals[0]])),
            float(sym.N(sol[vals[1]])),
            float(sym.N(sol[vals[2]])),
            float(sym.N(sol[vals[3]])),
            float(sym.N(sol[vals[4]])),
            float(sym.N(sol[vals[5]])),
        ])


In [ ]:
def simulate_box_jack_impact(f, x0, tspan, dt, integrate):

    N = int((max(tspan)-min(tspan))/dt)
    x = np.copy(x0)
    tvec = np.linspace(min(tspan),max(tspan), N)
    xtraj = np.zeros((len(x0), N))
    tm = 0
    for i in range(N):
        tm = tm + dt
        (impact, num) = impact_condition_die_jack(x, phif)
        display(impact,num)
        if impact is True:
            x = impact_update_die_jack(x, impact_eqns[num], [xbdotplus,ybdotplus,thetabdotplus,
           xjdotplus,yjdotplus,thetajdotplus])
            xtraj[:, i]=integrate(f, x, dt, tm)
        else:
            xtraj[:, i]=integrate(f, x, dt, tm)
        x = np.copy(xtraj[:,i])
    return xtraj

In [ ]:
t = [0,10]
dt = 0.01
s0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0])


In [ ]:
traj = simulate_box_jack_impact(dyn, s0, t, dt, integrate)

False

None

array([ 0,  0,  0,  0,  0,  0,  0,  0, -2,  0,  0,  0])

0.01

array([ 0.        ,  0.        , -0.01      ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -1.99997663,  0.        ,
       -0.04905   ,  0.        ])

...

10.004999999999832

array([  -0.2070747 , -124.34431958,  -11.90155529,    0.60396787,
       -127.33881315,    1.28813551,    2.77162295,  -25.50023891,
         -1.02636844,   -8.08390027,  -23.67525317,  -24.25647784])

10.004999999999832

array([  -0.19321658, -124.47182077,  -11.90668714,    0.56354837,
       -127.45718941,    1.16685312,    2.77162295,  -25.50023891,
         -1.02339227,   -8.08390027,  -23.72430317,  -24.25647784])

10.009999999999831

In [ ]:
traj

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.48649042e-01, -2.20932813e-01, -1.93216583e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.23961816e+02, -1.24216818e+02, -1.24471821e+02],
       [-1.99996883e-02, -3.99984417e-02, -5.99957928e-02, ...,
        -1.18861002e+01, -1.18964235e+01, -1.19066871e+01],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.08390027e+00, -8.08390027e+00, -8.08390027e+00],
       [-9.81000000e-02, -1.96200000e-01, -2.94300000e-01, ...,
        -2.35281032e+01, -2.36262032e+01, -2.37243032e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.42564778e+01, -2.42564778e+01, -2.42564778e+01]])

In [ ]:
def animate_die_in_a_box(config_array,l=1,w=0.2,T=1):

    ################################
    # Imports required for animation. (leave this part)
    from plotly.offline import init_notebook_mode, iplot
    from IPython.display import display, HTML
    import plotly.graph_objects as go

    #######################
    # Browser configuration. (leave this part)
    def configure_plotly_browser_state():
        import IPython
        display(IPython.core.display.HTML('''
            <script src="/static/components/requirejs/require.js"></script>
            <script>
              requirejs.config({
                paths: {
                  base: '/static/base',
                  plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
                },
              });
            </script>
            '''))
    configure_plotly_browser_state()
    init_notebook_mode(connected=False)


    ###############################################
    # Getting data from pendulum angle trajectories.
    N = len(config_array[0])

    xbpts = config_array[0]
    ybpts = config_array[1]
    thetabpts = config_array[2]
    xjpts = config_array[3]
    yjpts = config_array[4]
    thetajpts = config_array[5]


    b1x_array = np.zeros(N, dtype=np.float32)
    b1y_array = np.zeros(N, dtype=np.float32)
    b2x_array = np.zeros(N, dtype=np.float32)
    b2y_array = np.zeros(N, dtype=np.float32)
    b3x_array = np.zeros(N, dtype=np.float32)
    b3y_array = np.zeros(N, dtype=np.float32)
    b4x_array = np.zeros(N, dtype=np.float32)
    b4y_array = np.zeros(N, dtype=np.float32)


    j1x_array = np.zeros(N, dtype=np.float32)
    j1y_array = np.zeros(N, dtype=np.float32)
    j2x_array = np.zeros(N, dtype=np.float32)
    j2y_array = np.zeros(N, dtype=np.float32)
    j3x_array = np.zeros(N, dtype=np.float32)
    j3y_array = np.zeros(N, dtype=np.float32)
    j4x_array = np.zeros(N, dtype=np.float32)
    j4y_array = np.zeros(N, dtype=np.float32)

    for t in range(N):
        gwbs = get_se3_np(xbpts[t], ybpts[t], thetabpts[t])
        gwjs = get_se3_np(xjpts[t], yjpts[t], thetajpts[t])

        b1 = gwbs.dot(np.array([Lb, Lb, 0, 1]))
        b1x_array[t] = b1[0]
        b1y_array[t] = b1[1]
        b2 = gwbs.dot(np.array([Lb, -Lb, 0, 1]))
        b2x_array[t] = b2[0]
        b2y_array[t] = b2[1]
        b3 = gwbs.dot(np.array([-Lb, -Lb, 0, 1]))
        b3x_array[t] = b3[0]
        b3y_array[t] = b3[1]
        b4 = gwbs.dot(np.array([-Lb, Lb, 0, 1]))
        b4x_array[t] = b4[0]
        b4y_array[t] = b4[1]


        j1 = gwjs.dot(np.array([Lj, 0, 0, 1]))
        j1x_array[t] = j1[0]
        j1y_array[t] = j1[1]
        j2 = gwjs.dot(np.array([0, -Lj, 0, 1]))
        j2x_array[t] = j2[0]
        j2y_array[t] = j2[1]
        j3 = gwjs.dot(np.array([-Lj, 0, 0, 1]))
        j3x_array[t] = j3[0]
        j3y_array[t] = j3[1]
        j4 = gwjs.dot(np.array([0, Lj, 0, 1]))
        j4x_array[t] = j4[0]
        j4y_array[t] = j4[1]
        ####################################
    # Axis limits.
    xm = -45
    xM = 45
    ym = -15
    yM = -60

    ###########################
    # Defining data dictionary.
    data=[dict(name = 'Box'),
          dict(name = 'Jack'),

        ]

    ################################
    # Preparing simulation layout.
    layout=dict(xaxis=dict(range=[xm, xM], autorange=False, zeroline=False,dtick=1),
                yaxis=dict(range=[ym, yM], autorange=False, zeroline=False,dtick=1),
                title='Jack in a Box',
                hovermode='closest',
                updatemenus= [{'type': 'buttons',
                               'buttons': [{'label': 'Play','method': 'animate',
                                            'args': [None, {'frame': {'duration': T, 'redraw': False}}]},
                                           {'args': [[None], {'frame': {'duration': T, 'redraw': False}, 'mode': 'immediate',
                                            'transition': {'duration': 0}}],'label': 'Pause','method': 'animate'}
                                          ]
                              }]
               )

    ########################################
    # Defining the frames of the simulation.
    frames=[dict(data=[
            dict(x=[b1x_array[k],b2x_array[k],b3x_array[k],b4x_array[k],b1x_array[k]],
                 y=[b1y_array[k],b2y_array[k],b3y_array[k],b4y_array[k],b1y_array[k]],
                 mode='lines',
                 line=dict(color='blue', width=3)
                 ),
            dict(x=[j1x_array[k],j2x_array[k],j3x_array[k],j4x_array[k],j1x_array[k]],
                 y=[j1y_array[k],j2y_array[k],j3y_array[k],j4y_array[k],j1y_array[k]],
                 mode='lines',
                 line=dict(color='green', width=3)
                 ),
                 ]) for k in range(N)]

    #######################################
    # Putting it all together and plotting.
    figure1=dict(data=data, layout=layout, frames=frames)
    iplot(figure1)

##############
# The animation:


In [ ]:
def get_se3_np(x, y, theta):

    return np.array([[np.cos(theta), -np.sin(theta), 0, x],
                     [np.sin(theta),  np.cos(theta), 0, y],
                     [            0,              0, 1, 0],
                     [            0,              0, 0, 1]])

In [ ]:
traj

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.48649042e-01, -2.20932813e-01, -1.93216583e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.23961816e+02, -1.24216818e+02, -1.24471821e+02],
       [-1.99996883e-02, -3.99984417e-02, -5.99957928e-02, ...,
        -1.18861002e+01, -1.18964235e+01, -1.19066871e+01],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.08390027e+00, -8.08390027e+00, -8.08390027e+00],
       [-9.81000000e-02, -1.96200000e-01, -2.94300000e-01, ...,
        -2.35281032e+01, -2.36262032e+01, -2.37243032e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.42564778e+01, -2.42564778e+01, -2.42564778e+01]])

In [ ]:
animate_die_in_a_box(traj)